In [13]:
import pandas as pd
import os

frameworks = ["baseline", "vllm", "exllama", "together"]
measurers = ["ttft", "tpot", "throughput"]
prompt_sizes = ["17_tokens", "191_tokens", "660_tokens"]
result_path = "results/"


def getFileName(framework, mesurer_name, prompt_size):
    return f"{framework}_{mesurer_name}_{prompt_size}"


def getResult(filename: str):
    # check if file exists
    if not os.path.isfile(f"{result_path}{filename}"):
        print(f"File {filename} does not exist")
        return [0]
    with open(f"{result_path}{filename}", "r") as f:
        lines = f.readlines()
        data = []
        for line in lines:
            if "Iteration" in line:
                data.append(
                    float(line.split(":")[1].replace("GB", "").strip()))
    return data


def getMeasurerFilePath(framework, mesurer_name, prompt_size):
    return f"{framework}/{framework}_{mesurer_name}_{prompt_size}.txt"


def getGPUUsageFilePath(framework, mesurer_name, prompt_size):
    return f"{framework}/{framework}_{mesurer_name}_{prompt_size}_gpu_usage"


def combineResults() -> pd.DataFrame:
    df = pd.DataFrame()
    df["framework"] = []
    df["measurer"] = []
    df["prompt_size"] = []
    df["iterations"] = []
    df["average"] = []
    df["max"] = []
    df["min"] = []
    df["std"] = []
    df["gpu_usage"] = []

    for framework in frameworks:
        for measurer in measurers:
            for prompt_size in prompt_sizes:
                data = getResult(getMeasurerFilePath(
                    framework, measurer, prompt_size))
                gpu_usage = getResult(getGPUUsageFilePath(
                    framework, measurer, prompt_size))
                average = sum(data) / len(data)
                max_value = max(data)
                min_value = min(data)
                std = sum([(x - average) ** 2 for x in data]) / len(data)
                df = pd.concat([df, pd.DataFrame({
                    "framework": framework,
                    "measurer": measurer,
                    "prompt_size": prompt_size,
                    "iterations": len(data),
                    "average": average,
                    "max": max_value,
                    "min": min_value,
                    "std": std,
                    "gpu_usage": sum(gpu_usage) / len(gpu_usage)
                }, index=[0])], ignore_index=True)

    return df


data = combineResults()
data.to_csv("combined_results.csv", index=False)

File vllm/vllm_tpot_17_tokens_gpu_usage does not exist
File vllm/vllm_tpot_191_tokens_gpu_usage does not exist
File vllm/vllm_tpot_660_tokens_gpu_usage does not exist
